# Scripts Mosaicos 2008

In [1]:
import ee
import geemap
import os
import folium

In [2]:
Map = geemap.Map()
Map2 = geemap.Map()
Map3 = geemap.Map()

## Área de Mosaicos

### Patagonia

In [3]:
work_dir = os.path.expanduser('../../Data/shapes/')
patagonia = os.path.join(work_dir, 'patagonia_nueva.shp')
patagonia_grande = geemap.shp_to_ee(patagonia)
Map.addLayer(patagonia_grande, {}, 'Patagonia')
Map

Map(center=[40, -100], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=HBox(children=(T…

In [4]:
work_dir = os.path.expanduser('../../Data/shapes/')
norte = os.path.join(work_dir, 'area_norte.shp')
patagonia_norte = geemap.shp_to_ee(norte)
Map.addLayer(patagonia_norte, {}, 'norte')

In [5]:
work_dir = os.path.expanduser('../../Data/shapes/')
centro = os.path.join(work_dir, 'area_centro.shp')
patagonia_centro = geemap.shp_to_ee(centro)
Map.addLayer(patagonia_centro, {}, 'centro')

In [6]:
work_dir = os.path.expanduser('../../Data/shapes/')
sur = os.path.join(work_dir, 'area_sur.shp')
patagonia_sur = geemap.shp_to_ee(sur)
Map.addLayer(patagonia_grande, {}, 'sur')

## Funciones

In [7]:
def addNDVI(image): 
    ndvi = image.normalizedDifference(['B4', 'B3']).rename('NDVI')
    return image.addBands(ndvi)

In [8]:
def addNDWI(image):
    ndwi = image.normalizedDifference(['B2','B4']).rename('NDWI')
    return image.addBands(ndwi)

In [9]:
def addNDGI(image):
    ndgi = image.normalizedDifference(['B2','B3']).rename('NDGI')
    return image.addBands(ndgi)

In [10]:
def addDate(image):
    img_date = ee.Date(image.date())
    img_date = ee.Number.parse(img_date.format('YYYYMMdd'))
    return image.addBands(ee.Image(img_date).rename('date').toInt())

In [11]:
def add_ee_layer(self, ee_image_object, vis_params, name):
  map_id_dict = ee.Image(ee_image_object).getMapId(vis_params)
  folium.raster_layers.TileLayer(
      tiles=map_id_dict['tile_fetcher'].url_format,
      attr='Map Data &copy; <a href="https://earthengine.google.com/">Google Earth Engine</a>',
      name=name,
      overlay=True,
      control=True
  ).add_to(self)

folium.Map.add_ee_layer = add_ee_layer

In [12]:
def Cloud50(image):
    scored = ee.Algorithms.Landsat.simpleCloudScore(image)
    return image.updateMask(scored.select(['cloud']).lt(50))  

In [13]:
def Cloud30(image):
    scored = ee.Algorithms.Landsat.simpleCloudScore(image)
    return image.updateMask(scored.select(['cloud']).lt(30)) 

In [14]:
def Agua(image):
    return image.updateMask(ndwi.lte(-0.1))

In [15]:
def Agua2005(image):
    return image.updateMask(ndwi2005.gte(0.1))

In [16]:
def Agua2007(image):
    return image.updateMask(ndwi2007.gte(0))

In [17]:
def Agua2008(image):
    return image.updateMask(ndwi2008.gte(0))

In [18]:
def Ndvi2005(image):
    return image.updateMask(ndvi2005.gt(0.089))

In [19]:
def Ndvi2007(image):
    return image.updateMask(ndvi2007.lt(0.2))

In [20]:
def Ndvi2008(image):
    return image.updateMask(ndvi2008.lt(0.2))

In [21]:
def Ndvi2(image):
    return image.updateMask(ndvi.gte(0.2))

### Funciones SUR

In [22]:
def Ndvi_sur(image):
    return image.updateMask(ndvisur.gt(0.04))

In [23]:
def contra_Ndvi_sur(image):
    return image.updateMask(ndvisur.lte(0.04))

In [24]:
def Agua_sur(image):
    return image.updateMask(ndwisur.gte(0.3))

In [25]:
def contra_Agua_sur(image):
    return image.updateMask(ndwisur.lt(0.3))

## Colecciones

### Verano 2008

In [26]:
start_date = '2008-10-01'
end_date = '2009-03-31'

l5_2008 = ee.ImageCollection('LANDSAT/LT05/C01/T1_TOA') \
    .filterBounds(patagonia_grande) \
    .filterDate(start_date, end_date)

In [27]:
vizParams = {
  'bands': ['B4', 'B3', 'B2'],
  'min': 0,
  'max': 0.5,
  'gamma': [0.95, 1.1, 1]
}

In [28]:
coleccion2008 = l5_2008.map(addNDVI).map(addNDWI).map(addNDGI).map(addDate)

#### Verano Sur 2008-2009-2010

In [29]:
start_date = '2007-10-01'
end_date = '2011-03-31'

l5_sur = ee.ImageCollection('LANDSAT/LT05/C01/T1_TOA') \
    .filterBounds(patagonia_grande) \
    .filterDate(start_date, end_date)

In [30]:
coleccionsur = l5_sur.map(addNDVI).map(addNDWI).map(addNDGI).map(addDate)

### Verano 2007

In [31]:
start_date = '2007-10-01'
end_date = '2008-03-31'

l5_2007 = ee.ImageCollection('LANDSAT/LT05/C01/T1_TOA') \
    .filterBounds(patagonia_grande) \
    .filterDate(start_date, end_date)

In [32]:
coleccion2007 = l5_2007.map(addNDVI).map(addNDWI).map(addNDGI).map(addDate)

### Verano 2005

In [33]:
start_date = '2005-10-01'
end_date = '2006-03-31'

l5_2005 = ee.ImageCollection('LANDSAT/LT05/C01/T1_TOA') \
    .filterBounds(patagonia_grande) \
    .filterDate(start_date, end_date)

In [34]:
coleccion2005 = l5_2005.map(addNDVI).map(addNDWI).map(addNDGI).map(addDate)

## Quality Mosaic NDVI

In [35]:
NDVI_2008 = coleccion2008.qualityMosaic('NDVI')
NDVI_2007 = coleccion2007.qualityMosaic('NDVI')
NDVI_2005 = coleccion2005.qualityMosaic('NDVI')

In [36]:
NDVI_sur = coleccionsur.qualityMosaic('NDVI')

In [37]:
visParams = {
  'bands': ['B4', 'B3', 'B2'],
  'min': 0,
  'max': 0.4,
}
Map.addLayer(NDVI_2008, visParams, 'Greenest 2008')
Map.addLayer(NDVI_2007, visParams, 'Greenest 2007')
Map.addLayer(NDVI_2005, visParams, 'Greenest 2005')
Map.addLayer(NDVI_sur, visParams, 'Greenest sur 2008-2010')
Map.centerObject(patagonia_grande, 6)
Map

Map(center=[-47.89147413528022, -68.82549165231002], controls=(WidgetControl(options=['position', 'transparent…

In [38]:
ndvi2008 = NDVI_2008.select('NDVI')
ndvi2007 = NDVI_2007.select('NDVI')
ndvi2005 = NDVI_2005.select('NDVI')
ndvisur = NDVI_sur.select('NDVI')
palette = ['#d73027', '#f46d43', '#fdae61', '#fee08b', '#d9ef8b', '#a6d96a', '#66bd63', '#1a9850']

In [39]:
Map.addLayer(ndvi2008, {'palette': palette}, 'NDVI_2008')
Map.addLayer(ndvi2007, {'palette': palette}, 'NDVI_2007')
Map.addLayer(ndvi2005, {'palette': palette}, 'NDVI_2005')
Map.addLayer(ndvisur,{'palette': palette}, 'NDVI_sur 2008-2010')
Map

Map(center=[-47.89147413528022, -68.82549165231002], controls=(WidgetControl(options=['position', 'transparent…

## Agua

### Verano 2008

In [40]:
aquast_2008 = coleccion2008.qualityMosaic('NDWI')
aquast_sur = coleccionsur.qualityMosaic('NDWI')

### Verano 2007

In [41]:
aquast_2007 = coleccion2007.qualityMosaic('NDWI')

### Verano 2005

In [42]:
aquast_2005 = coleccion2005.qualityMosaic('NDWI')

### Quality Mosaic NDWI

In [43]:
visParams = {
  'bands': ['B4', 'B3', 'B1'],
  'min': 0,
  'max': 0.4,
    'gamma': [0.95, 1.1, 1],
}

Map2.addLayer(aquast_2008, visParams, 'Agua 2008')
Map2.addLayer(aquast_2007, visParams, 'Agua 2007')
Map2.addLayer(aquast_2005, visParams, 'Agua 2005')
Map2.addLayer(aquast_sur, visParams, 'Agua sur 2008-2010')
Map2.centerObject(patagonia_grande, 6)
Map2

Map(center=[-47.89147413528022, -68.82549165231002], controls=(WidgetControl(options=['position', 'transparent…

In [44]:
ndwi2008 = aquast_2008.select('NDWI')
ndwi_viz = ['00FFFF', '0000FF']
ndwi2007 = aquast_2007.select('NDWI')
ndwi2005 = aquast_2005.select('NDWI')
ndwisur = aquast_sur.select('NDWI')
Map2.addLayer(ndwi2008, {'palette': ndwi_viz}, 'NDWI_2008')
Map2.addLayer(ndwi2007, {'palette': ndwi_viz}, 'NDWI_2007')
Map2.addLayer(ndwi2005, {'palette': ndwi_viz}, 'NDWI_2005')
Map2.addLayer(ndwisur, {'palette': ndwi_viz}, 'NDWI_sur 2008-2010')
Map2

Map(center=[-47.89147413528022, -68.82549165231002], controls=(WidgetControl(options=['position', 'transparent…

In [45]:
ndwi_rgb = ndwi2008.visualize(**{
    'min': 0.5,
    'max':1,
    'palette': ['00FFFF', '0000FF']
})

In [46]:
ndwi_mask = ndwi2008.updateMask(ndwi2008.gte(0.4))

In [47]:
map_ndwi_mask = folium.Map(location = [-45, -73], zoom_start = 5)
map_ndwi_mask.add_ee_layer(ndwi_mask, {} , 'NDWI sin tierra')
display(map_ndwi_mask)

In [48]:
ndwi_mask2008 = ndwi2008.updateMask(ndwi2008.gte(0.4))

In [49]:
map_ndwi_mask2008 = folium.Map(location = [-45, -73], zoom_start = 5)
map_ndwi_mask2008.add_ee_layer(ndwi_mask2008, {}, 'NDWI sin tierra')
display(map_ndwi_mask2008)

## Máscaras

### Verano 1996

#### Nubes 50

In [50]:
coleccion08 = coleccion2008.map(Cloud50)
greenest08 = coleccion08.qualityMosaic('NDVI')
map_2008 = folium.Map(location = [-45, -73], zoom_start = 5)
map_2008.add_ee_layer(greenest08, vizParams, 'Mapa sin Nubes')
display(map_2008)

#### Nubes 30

In [51]:
coleccion07 = coleccion2007.map(Cloud30)
greenest07 = coleccion07.qualityMosaic('NDVI')
map_2007 = folium.Map(location = [-45, -73], zoom_start = 5)
map_2007.add_ee_layer(greenest07, vizParams, 'Mapa sin Nubes')
display(map_2007)

#### Nubes Sur

In [52]:
coleccion_sur = coleccionsur.map(Cloud30)
greenestsur = coleccion_sur.qualityMosaic('NDVI')
map_sur = folium.Map(location = [-45, -73], zoom_start = 5)
map_sur.add_ee_layer(greenestsur, vizParams, 'Mapa sin Nubes')
display(map_sur)

In [53]:
coleccionsur2 = coleccionsur.map(Cloud50)
greenestsur2 = coleccionsur2.qualityMosaic('NDVI')
map_sur2 = folium.Map(location = [-45, -73], zoom_start = 5)
map_sur2.add_ee_layer(greenestsur2, vizParams, 'Mapa sin Nubes')
display(map_sur2)

In [54]:
coleccion_sur = coleccionsur.map(Cloud30)
coleccion_sur = coleccion_sur.map(Agua_sur)
aquastsur = coleccion_sur.qualityMosaic('NDWI')
map_sur = folium.Map(location = [-45, -73], zoom_start = 5)
map_sur.add_ee_layer(aquastsur, vizParams, 'Mapa sin Nubes')
display(map_sur)

In [55]:
coleccion_sur = coleccionsur.map(Cloud30)
coleccion_sur = coleccion_sur.map(Ndvi_sur)
tierrasur = coleccion_sur.qualityMosaic('NDVI')
map_sur = folium.Map(location = [-45, -73], zoom_start = 5)
map_sur.add_ee_layer(tierrasur, vizParams, 'Mapa sin Nubes')
display(map_sur)

#### Filtro de agua

In [56]:
coleccion05_ = coleccion2005.map(Cloud50)
coleccion05_ = coleccion05_.map(Agua2005)
greenest05_ = coleccion05_.qualityMosaic('NDVI')
map_2005_ = folium.Map(location = [-45, -73], zoom_start = 5)
map_2005_.add_ee_layer(greenest05_, vizParams, 'Mapa sin Nubes')
display(map_2005_)

In [57]:
agua_2005 = coleccion2005.map(Cloud50)
agua_2005 = agua_2005.map(Agua2005)
aquast05_ = agua_2005.qualityMosaic('NDWI')
map_agua2005 = folium.Map(location = [-45, -73], zoom_start = 5)
map_agua2005.add_ee_layer(aquast05_, vizParams, 'Mapa sin Nubes')
display(map_agua2005)

In [58]:
coleccion08_2 = coleccion2008.map(Cloud50)
greenest08_2 = coleccion08_2.qualityMosaic('NDVI')
map_2008_2 = folium.Map(location = [-45, -73], zoom_start = 5)
map_2008_2.add_ee_layer(greenest08_2, vizParams, 'Mapa sin Nubes')
display(map_2008_2)

#### Filtro NDVI

In [59]:
coleccion08_3 = coleccion2008.map(Cloud50)
coleccion08_3 = coleccion08_3.map(Ndvi2008)
greenest08_3 = coleccion08_3.qualityMosaic('NDVI')
map_2008_3 = folium.Map(location = [-45, -73], zoom_start = 5)
map_2008_3.add_ee_layer(greenest08_3, vizParams, 'Mapa sin Nubes')
display(map_2008_3)

In [60]:
coleccion_sur2 = coleccionsur.map(Cloud30)
coleccion_sur2 = coleccion_sur2.map(Ndvi_sur)
coleccion_sur2 = coleccion_sur2.map(Agua_sur)
greenest_sur2 = coleccion_sur2.qualityMosaic('NDVI')
map_sur2 = folium.Map(location = [-45, -73], zoom_start = 5)
map_sur2.add_ee_layer(greenest_sur2, vizParams, 'Mapa sin Nubes')
display(map_sur2)

### Verano 2007

In [61]:
coleccion07 = coleccion2007.map(Cloud50)

greenest07 = coleccion07.qualityMosaic('NDVI')
map_2007 = folium.Map(location = [-45, -73], zoom_start = 5)
map_2007.add_ee_layer(greenest07, vizParams, 'Mapa sin nubes 07')
display(map_2007)

In [62]:
agua_2007 = coleccion2007.map(Agua2007)
aquast07_ = agua_2007.qualityMosaic('NDWI')
map_agua2007 = folium.Map(location = [-45, -73], zoom_start = 5)
map_agua2007.add_ee_layer(aquast07_, vizParams, 'Mapa sin Nubes 97')
display(map_agua2007)

### Verano 2008

In [63]:
coleccion08 = coleccion2008.map(Cloud50)
# coleccion2019 = coleccion2019.map(Agua)
greenest08 = coleccion08.qualityMosaic('NDVI')
map_2008 = folium.Map(location = [-45, -73], zoom_start = 5)
map_2008.add_ee_layer(greenest08, vizParams, 'Mapa sin nubes')
display(map_2008)

### Verano 2010

In [64]:
start_date = '2010-10-01'
end_date = '2011-03-31'

l5_2010 = ee.ImageCollection('LANDSAT/LT05/C01/T1_TOA') \
    .filterBounds(patagonia_grande) \
    .filterDate(start_date, end_date)


In [65]:
coleccion2010 = l5_2010.map(addNDVI).map(addNDWI).map(addNDGI).map(addDate)

In [66]:
coleccion10 = coleccion2010.map(Cloud30)
# coleccion2019 = coleccion2019.map(Agua)
greenest10 = coleccion10.qualityMosaic('NDVI')
map_2010 = folium.Map(location = [-45, -73], zoom_start = 5)
map_2010.add_ee_layer(greenest10, vizParams, 'Mapa sin nubes')
display(map_2010)


### Verano 2008 L7

In [90]:
start_date = '2008-10-01'
end_date = '2009-03-31'

l7_2008 = ee.ImageCollection('LANDSAT/LE07/C01/T1_TOA') \
    .filterBounds(patagonia_grande) \
    .filterDate(start_date, end_date)

In [91]:
coleccion2008l7 = l7_2008.map(addNDVI).map(addNDWI).map(addNDGI).map(addDate)

In [92]:
coleccion08l7 = coleccion2008l7.map(Cloud30)
# coleccion2008 = coleccion2008l7.map(Agua)
greenest08l7 = coleccion08l7.qualityMosaic('NDVI')
map_2008l7 = folium.Map(location = [-45, -73], zoom_start = 5)
map_2008l7.add_ee_layer(greenest08l7, vizParams, 'Mapa sin nubes')
display(map_2008l7)

### Verano 2009 l7

In [96]:
start_date = '2009-10-01'
end_date = '2010-03-31'

l7_2009 = ee.ImageCollection('LANDSAT/LE07/C01/T1_TOA') \
    .filterBounds(patagonia_grande) \
    .filterDate(start_date, end_date)



In [97]:
coleccion2009l7 = l7_2009.map(addNDVI).map(addNDWI).map(addNDGI).map(addDate)

In [98]:
coleccion09l7 = coleccion2009l7.map(Cloud30)
# coleccion2019 = coleccion2019.map(Agua)
greenest09l7 = coleccion09l7.qualityMosaic('NDVI')
map_2009l7 = folium.Map(location = [-45, -73], zoom_start = 5)
map_2009l7.add_ee_layer(greenest09l7, vizParams, 'Mapa sin nubes')
display(map_2009l7)

### Verano sur 2007-2010 L7

In [99]:
start_date = '2007-10-01'
end_date = '2010-03-31'

l7_sur = ee.ImageCollection('LANDSAT/LE07/C01/T1_TOA') \
    .filterBounds(patagonia_grande) \
    .filterDate(start_date, end_date)

In [100]:
coleccionl7sur = l7_sur.map(addNDVI).map(addNDWI).map(addNDGI).map(addDate)

In [102]:
coleccionl7sur = coleccionl7sur.map(Cloud30)
# coleccion2019 = coleccion2019.map(Agua)
greenestl7sur = coleccionl7sur.qualityMosaic('NDVI')
map_l7sur = folium.Map(location = [-45, -73], zoom_start = 5)
map_l7sur.add_ee_layer(greenestl7sur, vizParams, 'Mapa sin nubes')
display(map_l7sur)

## Mosaicos

In [76]:
Map3.addLayer(ndvi2008, {'palette': palette}, 'NDVI_2008')
Map3

Map(center=[40, -100], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=HBox(children=(T…

In [103]:
mosaic = ee.ImageCollection([greenestl7sur,greenestsur]).mosaic()

In [104]:
Map3.addLayer(mosaic, vizParams, 'Mosaico')
Map3

Map(bottom=11289.0, center=[-52.227799418870696, -68.78540039062501], controls=(WidgetControl(options=['positi…

In [79]:
mosaic2 = ee.ImageCollection([mosaic, tierrasur]).mosaic()

In [80]:
Map3.addLayer(mosaic2, vizParams, 'Mosaico2')
Map3

Map(center=[40, -100], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=HBox(children=(T…

In [81]:
# mosaic3 = ee.ImageCollection([mosaic2, greenest97]).mosaic()

In [82]:
# Map3.addLayer(mosaic3, vizParams, 'Mosaico3')
# Map3

In [83]:
# mosaic4 = ee.ImageCollection([mosaic3, greenest_sur2]).mosaic()

In [84]:
# Map3.addLayer(mosaic4, vizParams, 'Mosaico 4')
# Map3

In [85]:
# mosaic5 = ee.ImageCollection([mosaic4,aquast97_]).mosaic()

In [86]:
# Map3.addLayer(mosaic5, vizParams, 'Mosaico 5')
# Map3